In [1]:
import pydeck as pdk
import pandas as pd
import mercury as mr


In [2]:
demo = mr.Select(label="Select demo", choices=["ArcLayer", "GeoJsonLayer", "HexagonLayer"])

mercury.Select

In [3]:
if demo.value == "ArcLayer":

    DATA_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/geojson/vancouver-blocks.json"
    LAND_COVER = [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]]

    INITIAL_VIEW_STATE = pdk.ViewState(latitude=49.254, longitude=-123.13, zoom=11, max_zoom=16, pitch=45, bearing=0)

    polygon = pdk.Layer(
        "PolygonLayer",
        LAND_COVER,
        stroked=False,
        # processes the data as a flat longitude-latitude pair
        get_polygon="-",
        get_fill_color=[0, 0, 0, 20],
    )

    geojson = pdk.Layer(
        "GeoJsonLayer",
        DATA_URL,
        opacity=0.8,
        stroked=False,
        filled=True,
        extruded=True,
        wireframe=True,
        get_elevation="properties.valuePerSqm / 20",
        get_fill_color="[255, 255, properties.growth * 255]",
        get_line_color=[255, 255, 255],
    )

    r = pdk.Deck(layers=[polygon, geojson], initial_view_state=INITIAL_VIEW_STATE)


In [4]:
if demo.value == "GeoJsonLayer":
    DATA_URL = "https://raw.githubusercontent.com/ajduberstein/sf_public_data/master/bay_area_commute_routes.csv"
    # A bounding box for downtown San Francisco, to help filter this commuter data
    DOWNTOWN_BOUNDING_BOX = [
        -122.43135291617365,
        37.766492914983864,
        -122.38706428091974,
        37.80583561830737,
    ]


    def in_bounding_box(point):
        """Determine whether a point is in our downtown bounding box"""
        lng, lat = point
        in_lng_bounds = DOWNTOWN_BOUNDING_BOX[0] <= lng <= DOWNTOWN_BOUNDING_BOX[2]
        in_lat_bounds = DOWNTOWN_BOUNDING_BOX[1] <= lat <= DOWNTOWN_BOUNDING_BOX[3]
        return in_lng_bounds and in_lat_bounds


    df = pd.read_csv(DATA_URL)
    # Filter to bounding box
    df = df[df[["lng_w", "lat_w"]].apply(lambda row: in_bounding_box(row), axis=1)]

    GREEN_RGB = [0, 255, 0, 40]
    RED_RGB = [240, 100, 0, 40]

    # Specify a deck.gl ArcLayer
    arc_layer = pdk.Layer(
        "ArcLayer",
        data=df,
        get_width="S000 * 2",
        get_source_position=["lng_h", "lat_h"],
        get_target_position=["lng_w", "lat_w"],
        get_tilt=15,
        get_source_color=RED_RGB,
        get_target_color=GREEN_RGB,
        pickable=True,
        auto_highlight=True,
    )

    view_state = pdk.ViewState(
        latitude=37.7576171,
        longitude=-122.5776844,
        bearing=45,
        pitch=50,
        zoom=8,
    )


    TOOLTIP_TEXT = {"html": "{S000} jobs <br /> Home of commuter in red; work location in green"}
    r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT)

In [5]:
if demo.value == "HexagonLayer":
    HEXAGON_LAYER_DATA = (
        "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/3d-heatmap/heatmap-data.csv"  # noqa
    )

    # Define a layer to display on a map
    layer = pdk.Layer(
        "HexagonLayer",
        HEXAGON_LAYER_DATA,
        get_position=["lng", "lat"],
        auto_highlight=True,
        elevation_scale=50,
        pickable=True,
        elevation_range=[0, 3000],
        extruded=True,
        coverage=1,
    )

    # Set the viewport location
    view_state = pdk.ViewState(
        longitude=-1.415,
        latitude=52.2323,
        zoom=6,
        min_zoom=5,
        max_zoom=15,
        pitch=40.5,
        bearing=-27.36,
    )

    # Render
    r = pdk.Deck(layers=[layer], initial_view_state=view_state)

In [6]:
r

{
  "initialViewState": {
    "bearing": 0,
    "latitude": 49.254,
    "longitude": -123.13,
    "maxZoom": 16,
    "pitch": 45,
    "zoom": 11
  },
  "layers": [
    {
      "@@type": "PolygonLayer",
      "data": [
        [
          [
            -123.0,
            49.196
          ],
          [
            -123.0,
            49.324
          ],
          [
            -123.306,
            49.324
          ],
          [
            -123.306,
            49.196
          ]
        ]
      ],
      "getFillColor": [
        0,
        0,
        0,
        20
      ],
      "getPolygon": "@@=-",
      "id": "c1b34400-6a2f-471d-b504-02a109943f55",
      "stroked": false
    },
    {
      "@@type": "GeoJsonLayer",
      "data": "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/geojson/vancouver-blocks.json",
      "extruded": true,
      "filled": true,
      "getElevation": "@@=properties.valuePerSqm / 20",
      "getFillColor": "@@=[255, 255, properties.growth * 255]",
      "getLineColor": [
        255,
        255,
        255
      ],
      "id": "e3ebebba-2806-4bf8-937a-e2ca8b4eeeb9",
      "opacity": 0.8,
      "stroked": false,
      "wireframe": true
    }
  ],
  "mapProvider": "carto",
  "mapStyle": "https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json",
  "views": [
    {
      "@@type": "MapView",
      "controller": true
    }
  ]
}